In [6]:
import folium
import xarray as xr
import numpy as np
from pathlib import Path

# ==========================================
# CONFIGURATION
# ==========================================
DATA_FILE = "./surf_data_2024.nc"
def get_grid_coordinates(filepath):
    """
    Extracts just the Latitude and Longitude arrays.
    """
    path = Path(filepath)
    if not path.exists():
        print("⚠️ File not found. Using SYNTHETIC GRID (0.25° spacing).")
        # Create dummy grid matching ERA5 standard
        lats = np.arange(5.0, 10.25, 0.25)
        lons = np.arange(78.0, 83.25, 0.25)
        return lats, lons

    try:
        ds = xr.open_dataset(path)
        # We don't care about data values here, just the coordinates
        lats = ds.latitude.values
        lons = ds.longitude.values
        return lats, lons
    except Exception as e:
        print(f"Error reading NetCDF: {e}")
        return None, None

def create_raw_grid_map():
    print("Extracting Coordinates...")
    lats, lons = get_grid_coordinates(DATA_FILE)
    
    if lats is None: return

    # 1. Initialize Map
    center_lat = np.mean(lats)
    center_lon = np.mean(lons)
    m = folium.Map(location=[center_lat, center_lon], zoom_start=8, control_scale=True)

    # 2. Add Satellite Base Map
    folium.TileLayer(
        tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr='Esri',
        name='Satellite (Esri)',
        overlay=False,
        control=True
    ).add_to(m)

    # 3. Create Layer for Grid Points
    grid_layer = folium.FeatureGroup(name="ERA5 Raw Grid Nodes", show=True)
    
    print(f"Plotting {len(lats) * len(lons)} grid points...")
    
    # 4. Loop through ALL points (No masking)
    for lat in lats:
        for lon in lons:
            # UNIFORM BLUE STYLE
            folium.CircleMarker(
                location=[lat, lon],
                radius=3,              # Small radius to prevent clutter
                color='#3388ff',       # Border: Blue
                weight=1,
                fill=True,
                fill_color='#3388ff',  # Fill: Blue
                fill_opacity=0.6,
                popup=f"<b>Grid Node</b><br>Lat: {lat}<br>Lon: {lon}",
                tooltip=f"Node ({lat}, {lon})"
            ).add_to(grid_layer)

    # 6. Add Layers & Save
    grid_layer.add_to(m)
    folium.LayerControl(collapsed=False).add_to(m)

    output_file = "interactive_raw_grid.html"
    m.save(output_file)
    print(f"✅ Map Generated: {output_file}")
    print("   This map shows the pure spatial resolution of the dataset.")

if __name__ == "__main__":
    create_raw_grid_map()

Extracting Coordinates...
Plotting 441 grid points...
✅ Map Generated: interactive_raw_grid.html
   This map shows the pure spatial resolution of the dataset.
